# Guide

In this notebook we will create an interactive map of the UK which displays the % of Trans men in each local authority.

There are 331 local authorities in the UK, and we are using data collected in the 2021 UK Census which included 2 new questions on sexuality and gender identity.

The following data used are:

-   [Gender identity (detailed)](https://www.ons.gov.uk/datasets/TS070/editions/2021/versions/3) - this dataset classifies usual residents aged 16 years and over in England and Wales by gender identity.
-   [Local Authority District Boundaries](https://geoportal.statistics.gov.uk/datasets/bb53f91cce9e4fd6b661dc0a6c734a3f_0/about) - this file contains the digital vector boundaries for Local Authority Districts in the UK as of May 2022.

To create this visualisation we'll be using the Folium package that is used to create interactive leaflet maps. 

**NOTE:** If you're not following along with Binder, and you have your own computational environment, make sure you install the necessary packages through the command line before proceeding to import.

# Install packages

Uncomment the lines below to install the packages if you're not working in Binder.

In [ ]:
# !pip install pandas
# !pip install geopandas
# !pip install folium
# !pip install branca

# Import packages

In [ ]:
# Allows us to read-in csv files, and used for data manipulation
import pandas as pd
# Used to read-in and manipulate geospatial data
import geopandas as gpd
# Used to create interactive maps
import folium
# # Provides colormap support for Folium 
# import branca.colormap as cm

import os
import requests
import shutil
from zipfile import ZipFile

# Read-in dataset

In [ ]:
# First let's read in our gender identity dataset

df = pd.read_csv('Data/GI_det.csv')

In [ ]:
# Brief glimpse of data structure

df.head(10)

# Data Cleaning

Before we can calculate the %'s of trans men in each local authority, it's good to do some housekeeping and get our dataframe in order.

There's a few things that need sorting including:

1.  renaming columns so they are easier to reference
2.  removing 'Does not apply' from gender identity category


In [ ]:
# Use rename function and in columns parameter specify columns to rename
# Columns must be wrapped in a dictionary - {} i.e., the curly brackets

df = df.rename(columns = {'Lower tier local authorities Code': 'LA_code', 'Lower tier local authorities': 'LA_name', 'Gender identity (8 categories) Code': 'GI_code', 'Gender identity (8 categories)': 'GI_cat'})

In [ ]:
# Looks much better

df.head()

In [ ]:
# To get rid of the 'Does not apply' category we subset the dataframe...
# And we use a conditional '!=' to keep everything except 'Does not apply' category

df = df[df.GI_cat != 'Does not apply']

In [ ]:
# Unique function can be applied to a column in a dataframe to see which values are in that column
# 'Does not apply' has been succesfully dropped

df.GI_cat.unique()

# Data Pre-processing

Now onto the more interesting stuff. The data pre-processing stage involves preparing and transforming data into a suitable format for further analysis. It can involve selecting features, transforming variables, and creating new variables. For our purposes, we need to create a new column 'Percentages' which contains the % of Trans men in each local authority.

So, we'll need to first calculate the % of each gender identity category for each local authority. Then, we'll want to filter our dataset so that we only keep the responses related to Trans men.

In [ ]:
# Calculate percentage for each group and assign it directly to the 'Percentage' column
# Use the .groupby() function to group the data by local authority name and number of observations for that authority
# transform() takes a function and applies it to each group - i.e., applies lambda function to each group
# lambda() is a one-line function and for each group it divides individual observations by the sum for that local authority

df['Percentage'] = df.groupby('LA_name')['Observation'].transform(lambda x: round(x / x.sum() * 100, 2))

In [ ]:
# Let's check it out

df.head(10)

In [ ]:
# Now we can subset the dataframe so that we only keep the %s of trans men

df = df[df.GI_cat == 'Trans man'].reset_index(drop = True)

In [ ]:
df.head()

# Read-in shapefile

Now that we have our gender identity dataset sorted, we can start on the mapping process. And that starts with reading in our shapefile, which we should have downloaded from the geoportal. If (like me) you don't work with spatial data much, you might assume that you only need the shapefile, and you might delete the others that come with the folder. However, a shapefile is not just a single .shp file, but a collection of files that work together, and each of these files plays a crucial role in defining the shapefile's data and behaviour. When you try and read a shapefile into a coding editor, the software expects all components to be present, and missing them can lead to errors or incorrect spatial references. E.g. without the .dbf file, you'd lose all attribute data associated with the geographic features, and without the .shx file you might not be able to read the .shp file altogether. 

**TLDR: Make sure when you download the shapefile folder you keep all the files!**

Anyway, let's get started.

In [ ]:
# URL for the direct download of the shapefile
url = "https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/Local_Authority_Districts_May_2022_UK_BFE_V3_2022/FeatureServer/replicafilescache/Local_Authority_Districts_May_2022_UK_BFE_V3_2022_3331011932393166417.zip"

# Get the current working directory and construct the absolute path
current_dir = os.getcwd()
dest_dir = os.path.join(current_dir, "Shapefiles")
dest_file = os.path.join(dest_dir, "shapefile.zip")

# Create directory if it does not exist
os.makedirs(dest_dir, exist_ok=True)

# Download the shapefile
response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(dest_file, 'wb') as f:
        f.write(response.content)
    print("Download successful")
else:
    print("Failed to download the file, status code:", response.status_code)

# Unzip the file within the destination directory
with ZipFile(dest_file, 'r') as zip_ref:
    zip_ref.extractall(dest_dir)
    print("Files extracted to:", dest_dir)

In [ ]:
# Define the path to the actual shapefile (.shp)

shapefile_path = os.path.join(dest_dir, "LAD_MAY_2022_UK_BFE_V3.shp")

# Read the shapefile using geopandas
gdf = gpd.read_file(shapefile_path)

In [ ]:
# Let's check it out

gdf.head()

In [ ]:
# Inspect shape

gdf.shape

In [ ]:
# nunique() gives us the number of unique values in a column

gdf.LAD22NM.nunique()

# Clean shapefile

Hmm. We have 331 local authorities in our dataset that we want to plot, but there are 374 listed here.
We'll need to remove the local authorities that don't match the ones in our df.

* rename columns to match 'df'
* correct some LA names
* get rid of redundant Local Authorities

In [ ]:
# Use rename so gdf columns match those in original df

gdf = gdf.rename(columns = {'LAD22CD': 'LA_code', 'LAD22NM' : 'LA_name'})

In [ ]:
# Let's see if it worked

gdf.columns

In [ ]:
# Replace specific values in the LA_name column using .replace()

gdf['LA_name'] = gdf['LA_name'].replace({'Bristol, City of': 'Bristol', 'Kingston upon Hull, City of': 'Kingston upon Hull', 'Herefordshire, County of': 'Herefordshire'})

In [ ]:
# Subset the data and use .isin() to only keep local authorities that match those in df
# Reset the index after to avoid messing it up

gdf = gdf[gdf['LA_code'].isin(df.LA_code.unique())].reset_index(drop = True)

In [ ]:
gdf.shape

In [ ]:
# Nice

gdf.LA_code.nunique()

In [ ]:
type(gdf)

# Pre-processing shapefile

When it comes to mapping our data, it is important that we know which Coordinate Reference System (CRS) we are working with. Simply put, the CRS is a way to describe how the spatial data in the 'geodataframe' maps to locations on earth. The CRS is just a way of translating 3D reality into 2D maps. 

And when it comes to using mapping libraries like Folium, knowing the CRS is important because Folium expects coordinates in a specific format (usually latitude and longitude), which is EPSG:4326. If our CRS isn't in this format then we might need to transform it so that it matches what leaflet expects. Let's go ahead and see what our CRS is saying. 

In [ ]:
gdf.crs

In [ ]:
# Transform the GeoDataFrame to EPSG:4326 to get latitude and longitude
gdf = gdf.to_crs("EPSG:4326")

In [ ]:
gdf.crs

# Merge datasets

What we want to do now is merge our 'df' dataframe with our 'sf' spatial object, so that we can directly access the data and map it!

When you use the merge function in Python, the order in which you place the data matters in terms of the result's class type and spatial attributes. 
So, in terms of class type, we have a dataframe and a geodataframe. By placing 'gdf' first, the result will be a geodataframe, which is important because this retains the spatial characteristics and geometry columns of the geodataframe. We merge the columns on the LA_code and LA_name columns which are present in both datasets. 

In [ ]:
merged = pd.merge(gdf, df, on = ['LA_code', 'LA_name'], how = 'left')

In [ ]:
merged.head()

# Data Analysis

Finally, we can now build our interactive map using Folium. You can see from the 'geometry' column that we're working with 'MULTIPOLYGON'S' (a collection of polygons grouped together as a single geometric entitty), and 'POLYGONS'. In total we have 331 to plot, each representing a local authority. You can take a look at these separate multipolygons and polygons by using the .iloc() function and indexing the row and column (see below). 

In [ ]:
# use iloc() to select rows and columns by their position (index)

gdf.geometry.iloc[0]

In [ ]:
gdf.geometry.iloc[30]

## Building our interactive map

This approach has been adapted from GitHub user 'vverde' who has a brilliant guide to plotting with Python and Folium.
Link available here: 
https://vverde.github.io/blob/interactivechoropleth.html


## Step 1: Centering the map

The first thing we need to do is provide some coordinates so we can center our map on the UK.
Chat-GPT advised me that a commonly used central point for the UK is latitude 54.7, longitude -3.0. 

In [ ]:
# Define variables for latitude and longitude for the center of the UK

lat = 54.7
lon = -3.0

# Step 2: Initialise the map

When creating a Map in Folium, you generally specify which map tile you'd like to use, and theres a ton to choose from including:

* OpenStreetMap
* CartoDB
* Mapbox 

But, we're going to break convention and set our tiles to None. According to 'vverde', if we set a tile when we initialise the map, we are then unable to hide it from the LayerControl, which is what we want to achieve later on.

In [ ]:
# Initialise map by specifying location and how zoomed in you want it to be
# Tiles will be set to None

uk_map = folium.Map(location=[lat, lon], zoom_start=6, tiles = None)

In [ ]:
# Yep. Trust the process, we're going to add a Tile layer in the cell below.

uk_map

In [ ]:
# We're now going to add a layer of tiles over our map - in this case CartoDB positron
# We assign the name 'Light map' to our tile layer
# Control is set to False as we don't want users to be able to toggle the visibility of this layer

folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(uk_map)

uk_map

## Step 3: Draw choropleth

Now that we've set up our map with our base CartoDB positron tiles, we can now use Folium's choropleth function to plot our geometries onto the uk_map. The function below has a lot of parameters that we need to specify, including:

* geo_data = this refers to the GeoDataFrame which contains the geographic boundaries of each local authority.
* name = sets the name for this layer which we're placing on top of our base layer
* data = refers to the dataframe that contains the statistical data we'd like to plot
* columns = specifies the columns in our dataframe that Folium will use to match the geographic areas to their corresponding data values.
* key_on = tells Folium how to match the data in 'columns' with the geographic areas in geo_data. 'feature.properties.LA_code' indicates that Folium should look for a property named 'LA_code' in the GeoJSON representation of each area.

### ...What's a GeoJSON?

When we pass our GeoDataFrame to Folium (for example, as the geo_data parameter in the choropleth method), Folium internally converts this geographic information into GeoJSON. A GeoJSON is a format for encoding geographic data structures using JSON (JavaScript Object Notation). This conversion allows Folium to understand and render the geographic data on the map. 

Okay, let's look at the last few parameters:

* fill_color = sets the colour scheme, in our case 'YIGnBu' stands for Yellow-Green-Blue, which is a nice gradient for low to high values.
* fill_opacity and line_opacity = sets opacity of the area fill and boundary lines
* legend_name = labels the legend with string value
* smooth_factor = when this value is incremented it smooths the boundaries for each geographic shape, which can improve map loading and visual clarity

In [ ]:
folium.Choropleth(
 geo_data=merged,
 name='Choropleth',
 data=merged,
 columns=['LA_code','Percentage'],
 key_on="feature.properties.LA_code",
 fill_color='YlGnBu',
 fill_opacity=1,
 line_opacity=0.2,
 legend_name='% of Trans men',
 smooth_factor=0
).add_to(uk_map)

In [ ]:
uk_map

## Step 4: Add interactive elements

The following code adds interactivity to the map with custom styling for the map's features and tooltips which display specific information about a local authrotiy when a user scrolls over it.

In [ ]:
# Use anonymous lambda functions to define how each geographic area should be styled 

# this function defines how each area should be defined normally - i.e. when not being highlighted
# fillColor - fills areas with white colour, whilst  colour sets the border colour to black
# fillOpacity - defines how transparant the colour is
# weight - sets border thickness

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}

# this function defines how each area should appear when highlighted with mouse

highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}

The code below creates a GeoJson layer for the map, which allows us to add geographic data from our 'merged' GeoDataFrame with the specified styles and interactive features. Let's go through these parameters step by step:

* merged - specifies the GeoDataFrame containing the geographic data and attributes
* style_function = applies the styling to each geographic area, so border colour etc
* control = specifies that this layer shouldn't have a toggle option
* highlight_function = applies highlighting style when user hovers over area with mouse
* tooltip = defines a tooltip with relevant data that will be shown when user hovers over area

In [ ]:
features = folium.features.GeoJson(
    merged,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['LA_name','Percentage'],
        aliases=['LA name: ','% of Trans men: '],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)

In [ ]:
# Use add_child function to add interactivee layer to uk_map

uk_map.add_child(features)

# Adds a layer control widget to the map, allowing users to toggle the visibility of different layers
# So, if we set control = True, we can toggle our layers

folium.LayerControl().add_to(uk_map)


In [ ]:
uk_map

In [ ]:
# Get HTML representation of the map
map_html = uk_map._repr_html_()

# Save the HTML to a file (which you can manually copy to your markdown file later)
with open("Data/map.html", "w") as file:
    file.write(map_html)

In [ ]:
uk_map.save("Data/map2.html")

# Host map remotely